In [ ]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Eine Logelei

The following exercise is taken from the book 
<a href="https://www.amazon.de/Logeleien-Zweistein-ihren-Antworten-Wegner/dp/B006YF0VUE">"99 Logeleien von Zweistein"</a>.
This book has been published 1968.  It is written by 
<a href="http://de.wikipedia.org/wiki/Thomas_von_Randow">Thomas von Randow</a>.

---
The gentlemen Amann, Bemann, Cemann and Demann are called - not necessarily in the same order - by their first names Erich, Fritz, Gustav and Heiner. They are all married to exactly one woman. We also know the following about them and their wives:

- Either Amann's first name is Heiner, or Bemann's wife is Inge.
- If Cemann is married to Josefa, then - **and only in this case** - Klara's husband is **not** called Fritz.
- If Josefa's husband is **not** called Erich, then Inge is married to Fritz.
- If Luise's husband is called Fritz, then Klara's husband's first name is **not** Gustav.
- If the wife of Fritz is called Inge, then Erich is **not** married to Josefa.
- If Fritz is **not** married to Luise, then Gustav's wife's name is Klara.
- Either Demann is married to Luise, or Cemann is called Gustav.

*What are the full fullnames of these gentlemen, and what are their wives' first names?*

---

We are going to solve this problem by coding it in propositional logic and we will solve the resulting set of clauses using the Davis-Putnam algorithm.  In order to code the problem, we will use the following propositional variables:

- $\texttt{Name<}x\texttt{,}z\texttt{>}$ for any male first name $x$ and any surname $z$ expresses
  that the gentleman with first name $x$ has surname $z$.
- $\texttt{Ehe<}x\texttt{,}y\texttt{>}$ for any male first name $x$ and any female first name $y$ expresses
  that the gentleman with first name $x$ is married to the woman with first name $y$.

We are using the symbols $\texttt{<}$ and $\texttt{>}$ as part of the propositional variables because we want to show the structure of these variables and the parser for propositional logic accepts these symbols as part of propositional variables.

In [ ]:
Vornamen  = { "Erich",  "Fritz", "Gustav", "Heiner" }
Nachnamen = { "Amann", "Bemann", "Cemann", "Demann" }
Frauen    = { "Inge",  "Josefa", "Klara",  "Luise"  }

In [ ]:
%run ../Davis-Putnam.ipynb

The function $\texttt{makeVar}(f, x, y)$ creates a propositional variable of the form $\texttt{f<}x\texttt{,}y\texttt{>}$.

In [ ]:
def makeVar(f, x, y):
    return f + '<' + x + ',' + y + '>'

In [ ]:
makeVar('Ehe', 'Heiner', 'Klara')

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ computes a set of clauses expressing the fact that at most one of the variables of $S$ is <tt>True</tt>.

In [ ]:
def atMostOne(S): 
    return { frozenset({('¬',p), ('¬', q)}) for p in S
                                            for q in S 
                                            if  p != q 
           }

Given a set of propositional variables $S$, the function $\texttt{atLeastOne}(S)$ computes a set of clauses expressing the fact that at least one of the variables of $S$ is <tt>True</tt>.

In [ ]:
def atLeastOne(S):
    return { frozenset(S) }

$S$ is a set of propositional variables. The expression $\texttt{exactlyOne}(S)$ creates a set of clauses.  This set expresses the fact that exactly one of the variables in the set $S$ is true.

In [ ]:
def exactlyOne(S):
    return atMostOne(S) | atLeastOne(S)

For two sets $A$ and $B$ that have the same number of elements and a function symbol $f$, the procedure $\texttt{bijective}(A, B, f)$ computes a set of clauses that is equivalent to the formula
$$   \bigl(\forall x \in A: \exists! y \in B: f\langle x, y\rangle\bigr) \wedge
     \bigl(\forall y \in B: \exists! x \in A: f\langle x, y\rangle\bigr)
$$
Here the expression $f\langle x,y\rangle$ is the name of a propositional variable and the expression $\exists!x:p(x)$ is to be read as "There exists exactly one $x$ such that $p(x)$ holds".

In [ ]:
def bijective(A, B, f):
    "your code here"

For example, the function call `bijective({'a', 'b'}, {'x', 'y'}, 'f')` returns the following set of clauses:
```
{frozenset({('¬', 'f<b,x>'), ('¬', 'f<b,y>')}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<b,x>')}),
 frozenset({'f<a,y>', 'f<b,y>'}),
 frozenset({'f<b,x>', 'f<b,y>'}),
 frozenset({'f<a,x>', 'f<a,y>'}),
 frozenset({'f<a,x>', 'f<b,x>'}),
 frozenset({('¬', 'f<a,x>'), ('¬', 'f<a,y>')}),
 frozenset({('¬', 'f<a,y>'), ('¬', 'f<b,y>')})}
```

The function $\texttt{setToFormula}(S)$ converts the set of formulas $S$ from propositional logic into a conjunction of these formulas.  Since the formulas in $S$ can be complex, we have to ensure that everything is properly parenthesized.

In [ ]:
def setToFormula(S):
    if len(S) == 1:
        return '(' + S.pop() + ')'
    formula = S.pop()
    return f'({formula})' + ' ∧ ' + setToFormula(S)

In [ ]:
setToFormula({'a', 'b', 'c'})

The function $\texttt{isWifeOf}(y, z)$ returns a formula that is true if $y$ is the wife of $z$.  Here, $y$ is the first name of a woman, while $z$ is the last name of a man. The formula is returned as a string.  

In [ ]:
def isWifeOf(y, z):
    return setToFormula({f"Name<{x},{z}> → Ehe<{x},{y}>" for x in Vornamen })

In [ ]:
isWifeOf("Inge", "Amann")

In [ ]:
import propLogParser as plp

In [ ]:
run ../CNF.ipynb

The function $\texttt{parseAndNormalize}(s)$ takes a string $s$, parses this string as a propositional formula and then turns this formula into a set of clauses.

In [ ]:
def parseAndNormalize(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = normalize(nestedTuple)
    return Clauses

In [ ]:
parseAndNormalize('a ↔ ¬b')

The function `exclusiveOr(a, b)` computes the *exclusive or* of the formulas $a$ and $b$, which are given as strings. The resulting formula itself is converted into CNF.

In [ ]:
def exclusiveOr(a, b):
    formula     = f'(({a}) ↔ ¬({b}))'
    return parseAndNormalize(formula)

In [ ]:
exclusiveOr('p', 'q')

Below, you might need the following symbols: ¬, ∧, ∨, →, ↔

In [ ]:
def computeClauses():
    # Jedem männlichen Vornamen ist genau ein Nachname zugeordnet und umgekehrt.
    Clauses  = "your code here"
    # Jeder Mann ist mit genau einer Frau verheiratet und umgekehrt.
    Clauses |= "your code here"
    # Entweder ist Amanns Vorname Heiner, oder Bemanns Frau heisst Inge.
    Clauses |= "your code here"
    # Wenn Cemann mit Josefa verheiratet ist, dann – und nur in diesem Falle –
    # heisst Klaras Mann nicht Fritz.
    Clauses |= "your code here"
    # Wenn Josefas Mann nicht Erich heisst, dann ist Inge mit Fritz verheiratet.
    Clauses |= "your code here"
    # Wenn Luises Mann Fritz heisst, dann ist der Vorname von Klaras Mann nicht Gustav.
    Clauses |= "your code here"
    # Wenn die Frau von Fritz Inge heisst, dann ist Erich nicht mit Josefa verheiratet.
    Clauses |= "your code here"
    # Wenn Fritz nicht mit Luise verheiratet ist, dann heisst Gustavs Frau Klara.
    Clauses |= "your code here"
    # Entweder ist Demann mit Luise verheiratet, oder Cemann heisst Gustav.
    Clauses |= "your code here"
    return Clauses

In [ ]:
Clauses = computeClauses()
Clauses

There are 176 different clauses.

In [ ]:
len(Clauses)

In [ ]:
def compute_solution():
    Clauses = computeClauses()
    Result  = solve(Clauses, set())
    return Result

In [ ]:
Solution = compute_solution()

In [ ]:
import re

def extractFirst(s):
        m = re.search('<([A-Za-z]+),', s)
        return m.group(1)

def extractSecond(s):
        m = re.search(',([A-Za-z]+)>', s)
        return m.group(1)

In [ ]:
def displaySolution(Solution):
    Married = {}
    Names   = {}
    for Unit in Solution:
        for l in Unit:
            if isinstance(l, str):
                if l[:3] == "Ehe":
                    x = extractFirst(l)
                    y = extractSecond(l)
                    Married[x] = y
                elif l[:4] == "Name":
                    x = extractFirst(l)
                    y = extractSecond(l)
                    Names[x] = y
    for x in Married:
        print(f"{x} {Names[x]} is married to {Married[x]}.")

In [ ]:
displaySolution(Solution)

## Checking the Uniqueness of the Solution

In [ ]:
def arb(S):
    for x in S:
        return x

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [ ]:
def negateSolution(UnitClauses):
    return { complement(arb(unit)) for unit in UnitClauses }

In [ ]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

In [ ]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = solve(Clauses, set())
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [ ]:
checkUniqueness(Solution, Clauses)